### Perform Imports

In [ ]:
# import dependencies
import pandas as pd
import datetime
from os import listdir
from os.path import join, isfile
from random import sample
from numpy import nan

In [ ]:
# import eir functions
from eir_functions import clean_item_number, \
                          clean_drawing, \
                          clean_revision, \
                          clean_inspection_date, \
                          clean_inspector_operator, \
                          clean_disposition, \
                          clean_supplier, \
                          clean_receiver_number, \
                          clean_purchase_order, \
                          clean_job_order, \
                          clean_full_inspect_qty, \
                          clean_received_qty, \
                          clean_completed_qty

In [ ]:
# import confidential information
from sys import path
path.insert(0, "..")
from config import eir_cleaned_destination_csv

### Define Functions

In [ ]:
# this function prints out unique dataframe column values that contain certain values
def print_unique_values(df: pd.DataFrame, find_vals: list, show_cols: list = []) -> None:

    # define what columns are printed
    column_names = []
    if len(show_cols) > 0:
        column_names = show_cols
    else:
        column_names = df.columns

    # show all unique items in the column(s)
    for column in column_names:
        unique_list = [x for x in df[column].unique() if any(i in str(x) for i in find_vals)]
        nan_count = df[column].isna().sum()
        print("")
        print(f"----- {column}: {len(unique_list):,.0f} (NaN: {nan_count:,.0f}) -----")
        for item in unique_list:
            print(str(item))

### Build Functional Objects

In [ ]:
# metadata function object
meta_func_obj = {
    "item_number": {
        "func": clean_item_number,
        "args": {
            "none_if_contains": [],
            "remove_substrings": [" "],
            "replace_delimitors": ["\\", "/", "(", ")"]
        },
        "target_data_type": "string"
    },
    "drawing": {
        "func": clean_drawing,
        "args": {
            "none_if_contains": [" ", "."],
            "remove_substrings": [],
            "replace_delimitors": []
        },
        "target_data_type": "string"
    },
    "revision": {
        "func": clean_revision,
        "args": {
            "none_if_contains": [" ", "-", "/"],
            "remove_substrings": [],
            "replace_delimitors": []
        },
        "target_data_type": "string"
    },
    "inspection_date": {
        "func": clean_inspection_date,
        "args": {
            "none_if_contains": [],
            "remove_substrings": [],
            "replace_delimitors": []
        },
        "target_data_type": "datetime"
    },
    "inspector": {
        "func": clean_inspector_operator,
        "args": {
            "none_if_contains": [],
            "remove_substrings": [".", "(", ")", "{", "}", "[", "]", "<", ">"],
            "replace_delimitors": ["\\", "/", " ", "-", ","]
        },
        "target_data_type": "string"
    },
    "disposition": {
        "func": clean_disposition,
        "args": {
            "none_if_contains": [],
            "remove_substrings": [],
            "replace_delimitors": []
        },
        "target_data_type": "string"
    },
    "supplier": {
        "func": clean_supplier,
        "args": {
            "none_if_contains": [],
            "remove_substrings": [],
            "replace_delimitors": []
        },
        "target_data_type": "string"
    },
    "receiver_number": {
        "func": clean_receiver_number,
        "args": {
            "none_if_contains": ["no"],
            "remove_substrings": [" "],
            "replace_delimitors": ["-", "/", ","]
        },
        "target_data_type": "string"
    },
    "purchase_order": {
        "func": clean_purchase_order,
        "args": {
            "none_if_contains": ["no"],
            "remove_substrings": [" "],
            "replace_delimitors": ["-", "/", ","]
        },
        "target_data_type": "string"
    },
    "job_order": {
        "func": clean_job_order,
        "args": {
            "none_if_contains": [".", "-"],
            "remove_substrings": [" "],
            "replace_delimitors": []
        },
        "target_data_type": "string"
    },
    "operator": {
        "func": clean_inspector_operator,
        "args": {
            "none_if_contains": [],
            "remove_substrings": [".", "(", ")", "{", "}", "[", "]", "<", ">"],
            "replace_delimitors": ["\\", "/", " ", "-", ","]
        },
        "target_data_type": "string"
    },
    "full_inspect_qty": {
        "func": clean_full_inspect_qty,
        "args": {
            "none_if_contains": [],
            "remove_substrings": [" "],
            "replace_delimitors": []
        },
        "target_data_type": "float"
    },
    "received_qty": {
        "func": clean_received_qty,
        "args": {
            "none_if_contains": [" ", "/"],
            "remove_substrings": [],
            "replace_delimitors": []
        },
        "target_data_type": "float"
    },
    "completed_qty": {
        "func": clean_completed_qty,
        "args": {
            "none_if_contains": [" ", "/", "=", ".", "-"],
            "remove_substrings": [],
            "replace_delimitors": []
        },
        "target_data_type": "float"
    }
}

### Read Raw Data

In [ ]:
# read the two dataframes from csv
raw_metadata_df = pd.read_csv(join(eir_cleaned_destination_csv, "raw_metadata.csv"), low_memory = False)
raw_measurements_df = pd.read_csv(join(eir_cleaned_destination_csv, "raw_measurements.csv"), low_memory = False)

### Explore the Datasets

##### Show Unique Filtered Quantities

In [ ]:
# characters to view by
find_vals = [" ", "{", "}", "[", "]", "(", ")", "<", ">", "\\", "/", ",", "=", ".", "|", "-", "_"]

# toggle these comments to experiment with what special characters 'clutter' a particular column
# find_vals.remove(" ")
# find_vals.remove("{")
# find_vals.remove("}")
# find_vals.remove("[")
# find_vals.remove("]")
# find_vals.remove("(")
# find_vals.remove(")")
# find_vals.remove("<")
# find_vals.remove(">")
# find_vals.remove("\\")
# find_vals.remove("/")
# find_vals.remove(",")
# find_vals.remove("=")
# find_vals.remove(".")
# find_vals.remove("|")
# find_vals.remove("-")
# find_vals.remove("_")

# print the unique values that intersect with the find_vals list contents
print_unique_values(raw_metadata_df, find_vals, show_cols = [])

## Clean the Datasets

### Metadata

This cell turns all unwanted values into `None` for easier handling down the road.

In [ ]:
# create a deep copy of the raw dataframe
std_metadata_df = raw_metadata_df.copy(deep = True)

# apply the metadata function object to standardize 'unwanted' values
for k in meta_func_obj:

    # reference the object children
    my_func = meta_func_obj[k]["func"]
    my_args = meta_func_obj[k]["args"]

    if my_func is not None:
        std_metadata_df.loc[:, k] = raw_metadata_df[k].apply(my_func, args = (my_args,))

# characters to view by
find_vals = [" ", "{", "}", "[", "]", "(", ")", "<", ">", "\\", "/", ",", "=", ".", "|", "-"]

# toggle these comments to experiment with what special characters 'clutter' a particular column
# find_vals.remove(" ")
# find_vals.remove("{")
# find_vals.remove("}")
# find_vals.remove("[")
# find_vals.remove("]")
# find_vals.remove("(")
# find_vals.remove(")")
# find_vals.remove("<")
# find_vals.remove(">")
# find_vals.remove("\\")
# find_vals.remove("/")
# find_vals.remove(",")
# find_vals.remove("=")
# find_vals.remove(".")
# find_vals.remove("|")
# find_vals.remove("-")

# print the unique values that intersect with the find_vals list contents
print_unique_values(std_metadata_df, find_vals, show_cols = [])

This cell removes `None` values from certain columns.

In [ ]:
# record the row count before reduction
rc_initial = std_metadata_df.shape[0]

# create a reduced dataframe from the standardized dataframe
red_metadata_df = std_metadata_df.loc[
    (std_metadata_df["item_number"].isna() == False) & 
    (std_metadata_df["drawing"].isna() == False) & 
    (std_metadata_df["revision"].isna() == False), :
]

# record the row count after reduction
rc_reduced = red_metadata_df.shape[0]

# show how many rows were lost to the reduction
print(f"Rows Lost: {rc_initial:,.0f} to {rc_reduced:,.0f} ({rc_reduced - rc_initial:,.0f})")

This cell forces correct data types on the columns.

In [ ]:
# create a deep copy of the reduced dataframe
cln_metadata_df = red_metadata_df.copy(deep = True)

# preview the current data types
cln_metadata_df.dtypes

In [ ]:
# change data types
for k in meta_func_obj:
    target = meta_func_obj[k]["target_data_type"]
    if target == "datetime":
        cln_metadata_df[k] = pd.to_datetime(cln_metadata_df[k], format = "%Y-%m-%d")
    else:
        cln_metadata_df = cln_metadata_df.astype({ k: target })

# ensure the data types have changed
cln_metadata_df.dtypes